# Exploring Vaccine Temp WorkFlow

Notes:

Report ID = [38,39]

Objective:

    - export dataset from PID100 (above reports)
    - add a field ["redcap_event_name"]
    - enter uniform value into field: "march_2022_arm_1" "month_year_arm_1"
       - uniform format for redcap longitudinal event 
       - event must exist
    - export into dataset from PID278 



